In [1]:
from __future__ import division

import torch 
import torch.nn as nn
import torch.nn.functional as F 
from torch.autograd import Variable
import numpy as np
from util import *

In [65]:
def parse_cfg(cfgfile):
    """
    解析配置文件
    返回块列表，每个块代表神经网络中一个小单元，块定义为字典
    并用列表组织起来

    """
    
    file = open(cfgfile, 'r')
    lines = file.read().split('\n')                 # 把所有行馈入一个列表
    lines = [x for x in lines if len(x) > 0]        # 过滤空行 
    lines = [x for x in lines if x[0] != '#']       # 过滤注释
    lines = [x.rstrip().lstrip() for x in lines]    # 过滤左右空格
    
    block = {}
    blocks = []
    
    for line in lines:
        if line[0] == "[":               # 新块开始
            if len(block) != 0:          # 已经有块，追加
                blocks.append(block)     
                block = {}               # 初始化一个新块
            block["type"] = line[1:-1].rstrip() #填充块字典    
        else:
            key,value = line.split("=") 
            block[key.rstrip()] = value.lstrip()
    blocks.append(block)
    
    return blocks



In [66]:
class EmptyLayer(nn.Module):
    def __init__(self):
        super(EmptyLayer, self).__init__()
        

class DetectionLayer(nn.Module):
    def __init__(self, anchors):
        super(DetectionLayer, self).__init__()
        self.anchors = anchors

In [67]:
def create_modules(blocks):
    net_info = blocks[0]     
    module_list = nn.ModuleList()
    prev_filters = 3
    output_filters = []
    
    for index, x in enumerate(blocks[1:]):
        module = nn.Sequential()
    
        #检查块的类型
        #为每一个块建一个新的module
        #追加到module_list
        
        #卷积层
        if (x["type"] == "convolutional"):
            #获取层信息
            activation = x["activation"]
            try:
                batch_normalize = int(x["batch_normalize"])
                bias = False
            except:
                batch_normalize = 0
                bias = True
        
            filters= int(x["filters"])
            padding = int(x["pad"])
            kernel_size = int(x["size"])
            stride = int(x["stride"])
        
            if padding:
                pad = (kernel_size - 1) // 2
            else:
                pad = 0
        
            #装配卷积层
            conv = nn.Conv2d(prev_filters, filters, kernel_size, stride, pad, bias = bias)
            #print(type(conv))
            module.add_module("conv_{0}".format(index), conv)
        
            #装配批归一化层
            if batch_normalize:
                bn = nn.BatchNorm2d(filters)
                module.add_module("batch_norm_{0}".format(index), bn)
        
             
            #设置Leaky ReLU
            if activation == "leaky":
                activn = nn.LeakyReLU(0.1, inplace = True)
                module.add_module("leaky_{0}".format(index), activn)
        
        #上采样
        elif (x["type"] == "upsample"):
            stride = int(x["stride"])
            upsample = nn.Upsample(scale_factor = 2, mode = "nearest")
            module.add_module("upsample_{}".format(index), upsample)
                
        #路由层
        elif (x["type"] == "route"):
            x["layers"] = x["layers"].split(',')
            start = int(x["layers"][0])
            try:
                end = int(x["layers"][1])
            except:
                end = 0
            #转为路由层相对层距离
            if start > 0: 
                start = start - index
            if end > 0:
                end = end - index
            route = EmptyLayer()
            module.add_module("route_{0}".format(index), route)
            x["layers"][0]=start
            if end!=0:
                 x["layers"][1]=end
            #if end < 0:
            if len(x["layers"])==2:
                filters = output_filters[index + start] + output_filters[index + end]
            else:
                filters= output_filters[index + start]
    
        #快捷连接，也就是跳过连接
        elif x["type"] == "shortcut":
            shortcut = EmptyLayer()
            module.add_module("shortcut_{}".format(index), shortcut)
            
        #yolo层是检测层
        #mask = 0,1,2
        #anchors = 10,13,  16,30,  33,23,  ...
        elif x["type"] == "yolo":
            mask = x["mask"].split(",")
            mask = [int(x) for x in mask]
    
            anchors = x["anchors"].split(",")
            anchors = [int(a) for a in anchors]
            anchors = [(anchors[i], anchors[i+1]) for i in range(0, len(anchors),2)]
            anchors = [anchors[i] for i in mask]
            print("第{0}层:{1}".format(index,mask))
            print(anchors)
            
            detection = DetectionLayer(anchors)
            module.add_module("Detection_{}".format(index), detection)
                              
        module_list.append(module)
        prev_filters = filters
        output_filters.append(filters)
        
    return (net_info, module_list)

In [68]:
class Darknet(nn.Module):
    def __init__(self, cfgfile):
        super(Darknet, self).__init__()
        self.blocks = parse_cfg(cfgfile)
        #调用create_modules函数
        self.net_info, self.module_list = create_modules(self.blocks)
        
    def forward(self, x, CUDA):
        #获取所有的module，也就是块
        modules = self.blocks[1:]
        #存储outputs，用于快捷层和路由层
        outputs = {}   
        
        #是否遇到第一个检测
        write = 0
        for i, module in enumerate(modules):
            #块类型
            module_type = (module["type"])
            #直接调用块函数,输入x
            if module_type == "convolutional" or module_type == "upsample":
                x = self.module_list[i](x)
            #对于路由层，获取路由指向的层
            elif module_type == "route":
                layers = module["layers"]
                layers = [int(a) for a in layers]
    
                #只指向一个层的路由层，直接输出这个层即可
                print("--------------------------")
                print("当前路由层:",i)
                if len(layers) == 1:
                    x = outputs[i + (layers[0])]
                    print("路由源于第{0}层,特征图维度：{1}".format(i+layers[0],x.shape))

                else:
                    #指向两个层，串接
                    map1 = outputs[i + layers[0]]
                    map2 = outputs[i + layers[1]]
                    
                    print("第1个路由源于第{0}层,特征图维度：{1}".format(i+layers[0],map1.shape))
                    print("第1个路由源于第{0}层,特征图维度：{1}".format(i+layers[1],map2.shape))

                    x = torch.cat((map1, map2), 1)
                    print("路由层拼接后：",x.shape)
            #跳过连接只指向一个层，也就是从哪层跳过来
            elif  module_type == "shortcut":
                from_ = int(module["from"])
                #把上一层和指向的层加起来
                x = outputs[i-1] + outputs[i+from_]
            
            #如果是检测层
            elif module_type == 'yolo':        
                anchors = self.module_list[i][0].anchors
                #print("**",self.module_list[i])
                #从net_info(实际就是blocks[0]，即[net])获取输入尺度
                inp_dim = int (self.net_info["height"])
        
                #获取类别总数
                num_classes = int (module["classes"])
        
                #这是检测层的特征图 
                x = x.data
                print("第{0}层特征图维度:{1}".format(i,x.shape))
                
                #在util.py中的predict_transform()函数利用x
                #得到每个格子所对应的anchor最终得到的目标
                # 坐标与宽高，以及出现目标的得分与每种类别的得分
                #经过predict_transform变换后的x的维度是
                #(batch_size, grid_size*grid_size*num_anchors, 5+类别数量)
                x = predict_transform(x, inp_dim, anchors, num_classes, CUDA)
                if not write:              #if no collector has been intialised. 
                    detections = x
                    write = 1
                #这样3个yolo层的预测值按照每个方框对应的行的维度进行连接
                #得到了这张图处所有anchor的预测值，后面的NMS等操作可以一次完成

                else:       
                    detections = torch.cat((detections, x), 1)
        
            outputs[i] = x
        
        return detections


    def load_weights(self, weightfile):
        #Open the weights file
        fp = open(weightfile, "rb")
    
        #The first 5 values are header information 
        # 1. Major version number
        # 2. Minor Version Number
        # 3. Subversion number 
        # 4,5. Images seen by the network (during training)
        header = np.fromfile(fp, dtype = np.int32, count = 5)
        self.header = torch.from_numpy(header)
        self.seen = self.header[3]   
        
        weights = np.fromfile(fp, dtype = np.float32)
        
        ptr = 0
        for i in range(len(self.module_list)):
            module_type = self.blocks[i + 1]["type"]
    
            #If module_type is convolutional load weights
            #Otherwise ignore.
            
            if module_type == "convolutional":
                model = self.module_list[i]
                try:
                    batch_normalize = int(self.blocks[i+1]["batch_normalize"])
                except:
                    batch_normalize = 0
            
                conv = model[0]
                
                
                if (batch_normalize):
                    bn = model[1]
        
                    #Get the number of weights of Batch Norm Layer
                    num_bn_biases = bn.bias.numel()
        
                    #Load the weights
                    bn_biases = torch.from_numpy(weights[ptr:ptr + num_bn_biases])
                    ptr += num_bn_biases
        
                    bn_weights = torch.from_numpy(weights[ptr: ptr + num_bn_biases])
                    ptr  += num_bn_biases
        
                    bn_running_mean = torch.from_numpy(weights[ptr: ptr + num_bn_biases])
                    ptr  += num_bn_biases
        
                    bn_running_var = torch.from_numpy(weights[ptr: ptr + num_bn_biases])
                    ptr  += num_bn_biases
        
                    #Cast the loaded weights into dims of model weights. 
                    bn_biases = bn_biases.view_as(bn.bias.data)
                    bn_weights = bn_weights.view_as(bn.weight.data)
                    bn_running_mean = bn_running_mean.view_as(bn.running_mean)
                    bn_running_var = bn_running_var.view_as(bn.running_var)
        
                    #Copy the data to model
                    bn.bias.data.copy_(bn_biases)
                    bn.weight.data.copy_(bn_weights)
                    bn.running_mean.copy_(bn_running_mean)
                    bn.running_var.copy_(bn_running_var)
                
                else:
                    #Number of biases
                    num_biases = conv.bias.numel()
                
                    #Load the weights
                    conv_biases = torch.from_numpy(weights[ptr: ptr + num_biases])
                    ptr = ptr + num_biases
                
                    #reshape the loaded weights according to the dims of the model weights
                    conv_biases = conv_biases.view_as(conv.bias.data)
                
                    #Finally copy the data
                    conv.bias.data.copy_(conv_biases)
                    
                #Let us load the weights for the Convolutional layers
                num_weights = conv.weight.numel()
                
                #Do the same as above for weights
                conv_weights = torch.from_numpy(weights[ptr:ptr+num_weights])
                ptr = ptr + num_weights
                
                conv_weights = conv_weights.view_as(conv.weight.data)
                conv.weight.data.copy_(conv_weights)

In [69]:
blocks=parse_cfg("cfg/yolov3.cfg")
print(create_modules(blocks))

第82层:[6, 7, 8]
[(116, 90), (156, 198), (373, 326)]
第94层:[3, 4, 5]
[(30, 61), (62, 45), (59, 119)]
第106层:[0, 1, 2]
[(10, 13), (16, 30), (33, 23)]
({'type': 'net', 'batch': '1', 'subdivisions': '1', 'width': '416', 'height': '416', 'channels': '3', 'momentum': '0.9', 'decay': '0.0005', 'angle': '0', 'saturation': '1.5', 'exposure': '1.5', 'hue': '.1', 'learning_rate': '0.001', 'burn_in': '1000', 'max_batches': '500200', 'policy': 'steps', 'steps': '400000,450000', 'scales': '.1,.1'}, ModuleList(
  (0): Sequential(
    (conv_0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (batch_norm_0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (leaky_0): LeakyReLU(negative_slope=0.1, inplace)
  )
  (1): Sequential(
    (conv_1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (batch_norm_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (leaky_1): LeakyRe

In [70]:
def get_test_input():
    img = cv2.imread("dog-cycle-car.png")
    img = cv2.resize(img, (416,416))          #Resize to the input dimension
    img_ =  img[:,:,::-1].transpose((2,0,1))  # BGR -> RGB | H X W C -> C X H X W 
    img_ = img_[np.newaxis,:,:,:]/255.0       #Add a channel at 0 (for batch) | Normalise
    img_ = torch.from_numpy(img_).float()     #Convert to float
    img_ = Variable(img_)                     # Convert to Variable
    return img_

In [64]:
model=Darknet("cfg/yolov3.cfg")
model.load_weights("yolov3.weights")
inp=get_test_input()
pred=model(inp,torch.cuda.is_available())
print(pred.shape)
print(pred[0][0])

第82层:[6, 7, 8]
[(116, 90), (156, 198), (373, 326)]
第94层:[3, 4, 5]
[(30, 61), (62, 45), (59, 119)]
第106层:[0, 1, 2]
[(10, 13), (16, 30), (33, 23)]
** Sequential(
  (Detection_82): DetectionLayer()
)
第82层特征图维度:torch.Size([1, 255, 13, 13])
先验框:
框个数：3
第一个框的宽、高： (3.625, 2.8125)
torch.Size([1, 507, 85])
--------------------------
当前路由层: 83
路由源于第79层,特征图维度：torch.Size([1, 512, 13, 13])
--------------------------
当前路由层: 86
第1个路由源于第85层,特征图维度：torch.Size([1, 256, 26, 26])
第1个路由源于第61层,特征图维度：torch.Size([1, 512, 26, 26])
路由层拼接后： torch.Size([1, 768, 26, 26])
** Sequential(
  (Detection_94): DetectionLayer()
)
第94层特征图维度:torch.Size([1, 255, 26, 26])
先验框:
框个数：3
第一个框的宽、高： (1.875, 3.8125)
torch.Size([1, 2028, 85])
--------------------------
当前路由层: 95
路由源于第91层,特征图维度：torch.Size([1, 256, 26, 26])
--------------------------
当前路由层: 98
第1个路由源于第97层,特征图维度：torch.Size([1, 128, 52, 52])
第1个路由源于第36层,特征图维度：torch.Size([1, 256, 52, 52])
路由层拼接后： torch.Size([1, 384, 52, 52])


/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


** Sequential(
  (Detection_106): DetectionLayer()
)
第106层特征图维度:torch.Size([1, 255, 52, 52])
先验框:
框个数：3
第一个框的宽、高： (1.25, 1.625)
torch.Size([1, 8112, 85])
torch.Size([1, 10647, 85])
tensor([8.5426e+00, 1.9015e+01, 1.1130e+02, 7.6208e+01, 4.0703e-08, 3.7478e-01,
        3.2929e-04, 7.2857e-02, 3.6275e-04, 1.2435e-03, 5.8169e-03, 7.5506e-04,
        8.3784e-03, 2.0736e-03, 3.4834e-03, 1.5130e-03, 4.8598e-03, 1.6946e-03,
        2.5354e-03, 1.6692e-03, 5.5098e-04, 3.8665e-04, 8.7062e-04, 7.9639e-04,
        5.4608e-03, 2.8599e-03, 2.5922e-03, 3.4154e-03, 3.7444e-03, 5.8627e-03,
        8.1170e-02, 7.5729e-03, 4.0476e-03, 2.8710e-03, 1.6677e-03, 2.5688e-03,
        1.1758e-03, 2.3890e-03, 1.2358e-02, 4.9009e-03, 1.3348e-03, 3.9646e-04,
        2.7907e-03, 1.8299e-03, 1.2027e-03, 2.7431e-03, 2.1670e-03, 2.9131e-03,
        5.0090e-03, 3.4783e-03, 1.5091e-03, 3.8293e-03, 1.5713e-02, 6.3134e-04,
        6.2511e-03, 4.0254e-03, 9.7105e-03, 8.4440e-04, 2.8784e-03, 1.1807e-02,
        3.5873e-03,